*   **Año:** [2024]
*   **Alumno/a:** [Gianni Mana]
*   **Legajo:** [45155554]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [3]:
import pydantic as pyd

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [4]:
class Client(pyd.BaseModel):
    dni: pyd.conint(strict=True)
    nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [13]:
cliente = Client(dni = 39755010, nacionalidad = "Argentina")
print(f"El cliente posee DNI: {cliente.dni} y nacionalidad: {cliente.nacionalidad}")


El cliente posee DNI: 39755010 y nacionalidad: Argentina


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [14]:
try:
    cliente_fallo = Client(dni = "39755010", nacionalidad = "Argentina")
    print("Cliente inválido:")
    print(cliente_fallo)
except pyd.ValidationError as e:
    print("Error en la validación del cliente:")
    print(e)

Error en la validación del cliente:
1 validation error for Client
dni
  Input should be a valid integer [type=int_type, input_value='39755010', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_type


Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [33]:
from datetime import date
from typing import Literal

class Client(pyd.BaseModel):
    dni: pyd.conint(gt=0)
    nacionalidad: Literal["Argentina", "Uruguay", "Brasil"]
    fecha_registro: date

    @pyd.validator("fecha_registro")
    def check_fecha_registro(cls, a):
        if a > date.today():
            raise ValueError("La fecha de registro es invalida")
        return a

<ipython-input-33-641a7b90e05d>:9: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @pyd.validator("fecha_registro")


In [22]:
try:
    cliente = Client(dni = 45155554, nacionalidad = "Argentina", fecha_registro = date(2024, 11, 12))
    print("Cliente:")
    print(cliente)
except pyd.ValidationError as e:
    print("Error")
    print(e)

Cliente:
dni=45155554 nacionalidad='Argentina' fecha_registro=datetime.date(2024, 11, 12)


In [24]:
try:
    cliente_fallo = Client(dni = -202401201, nacionalidad = "Argentina", fecha_registro = date(2030, 10, 10))
    print("Cliente:")
    print(cliente_fallo)
except pyd.ValidationError as e:
    print("Error")
    print(e)

Error
2 validation errors for Client
dni
  Input should be greater than 0 [type=greater_than, input_value=-202401201, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than
fecha_registro
  Value error, La fecha de registro es invalida [type=value_error, input_value=datetime.date(2030, 10, 10), input_type=date]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [35]:
try:
    cliente = Client(dni = -45130906, nacionalidad = "Boliviana", fecha_registro = date(2032, 1, 13))
    print(f"Cliente: {cliente}")
except pyd.ValidationError as e:
    print("Error")
    print(e)

Error
3 validation errors for Client
dni
  Input should be greater than 0 [type=greater_than, input_value=-45130906, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than
nacionalidad
  Input should be 'Argentina', 'Uruguay' or 'Brasil' [type=literal_error, input_value='Boliviana', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
fecha_registro
  Value error, La fecha de registro es invalida [type=value_error, input_value=datetime.date(2032, 1, 13), input_type=date]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [ ]:
!pip install pandera

In [40]:
import pandera as pa
from datetime import date
import pandas as pd

schema = pa.DataFrameSchema(
    {
        "dni": pa.Column(int, checks=pa.Check.ge(1)),
        "nacionalidad": pa.Column(str, checks=pa.Check.isin(["Argentina", "Uruguay", "Brasil"])),
        "fecha_registro": pa.Column(
            pa.DateTime,
            checks=pa.Check(
                lambda fecha: fecha <= pd.Timestamp(date.today()),
                element_wise=True
            )
        ),
    }
)


dataframe = pd.DataFrame({
    "dni": [45155554],
    "nacionalidad": ["Argentina"],
    "fecha_registro": [pd.Timestamp("2003-12-11")]
})

try:
    schema.validate(df)
    print("El DataFrame es válido")
except pa.errors.SchemaError as e:
    print(f"Error de validación: {e}")

dataframe2 = pd.DataFrame({
    "dni": [-14890211],
    "nacionalidad": ["Uruguay"],
    "fecha_registro": [pd.Timestamp("2024-10-18")]
})


try:
    schema.validate(df2)
    print("El dataframe es válido")
except pa.errors.SchemaError as e:
    print(f"Error de validación: {e}")


dataframe3 = pd.DataFrame({
    "dni": [12345678],
    "nacionalidad": ["Boliviana"],
    "fecha_registro": [pd.Timestamp("2024-01-18")]
})

try:
    schema.validate(df3)
    print("El dataframe es válido")
except pa.errors.SchemaError as e:
    print(f"Error de validación: {e}")

El DataFrame es válido
Error de validación: Column 'dni' failed element-wise validator number 0: greater_than_or_equal_to(1) failure cases: -23456789
Error de validación: Column 'nacionalidad' failed element-wise validator number 0: isin(['Argentina', 'Uruguay', 'Brasil']) failure cases: Peru
